# Experiments with vectorized functions

## todo

- since these functions are now `nogil`, threaded parallelization
  should provide speedups on multicore systems. try it!

In [1]:
from h3._cy.geo import haversine_vect, geo_to_h3_vect
import h3.api.numpy_int as h3i
import h3.api.basic_str as h3s

import numpy as np

# `geo_to_h3` times

In [2]:
N = 1_000_000

lats = np.random.uniform(0, 90, N) 
lons = np.random.uniform(0, 90, N)

In [3]:
%%timeit
[h3s.geo_to_h3(lat, lon, 10) for lat, lon in zip(lats, lons)]

1.91 s ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
[h3i.geo_to_h3(lat, lon, 10) for lat, lon in zip(lats, lons)]

1.91 s ± 129 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
out = np.zeros(N, dtype='uint64')
geo_to_h3_vect(lats, lons, 9, out)

868 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Haversine distances between cells

compute haversine distances between h3 cells, using
integer representation (avoiding conversion to/from `str` representation)

In [6]:
def rand_hexes(N):
    lats = np.random.uniform(0, 90, N) 
    lons = np.random.uniform(0, 90, N)

    out = np.array(
        [h3i.geo_to_h3(a, o, 9) for a,o in zip(lats, lons)],
        dtype = 'uint64',
    )
    
    return out

In [7]:
N = 1_000_000

a = rand_hexes(N)
b = rand_hexes(N)

In [8]:
%%timeit
out = np.zeros(N, dtype='double')
haversine_vect(a,b,out)

1.02 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
